In [34]:
import sys
sys.path.insert(0, "../../../")
from clases.utils import setup_environment, connect_database, sys_anio, sys_mes_ini, sys_mes_fin

setup_environment()
conn = connect_database()
anio=sys_anio()
mes_ini=sys_mes_ini()
mes_fin=sys_mes_fin()



Conexion exitosa con la Base de datos:irvin_hisminsa


In [35]:
def gestante_captadas(conn,anio,mes_ini,mes_fin):
    gestante_captadas=conn.df(f"""
  SELECT 
    provincia,
    distrito,
    desc_ue,
    red,
    microred,
    nombre_eess,
    anio,
    mes,
    codigo_item,
    numero_documento ,
    cod_eess,
    fecha_atencion AS "fechacaptada",
   nt.valor_lab ,
   row_number() over (partition by mp.numero_documento,nt.id_establecimiento order by nt.anio,nt.fecha_atencion) as gestantes
FROM 
    maestros.nominal_trama nt  
    INNER JOIN maestros.maestro_paciente mp  ON nt.id_paciente = mp.id_paciente
    INNER JOIN maestros.eess_geresa_cusco egc ON nt.id_establecimiento = egc.id_eess
WHERE 
    nt.codigo_item LIKE ANY (ARRAY['Z349_', 'Z359_'])
    AND genero = 'F' 
    AND anio in ({anio}) 
    AND mes BETWEEN {mes_ini} AND {mes_fin} 
    AND id_tipo_documento = '1' 
    AND cat IN ('I-1', 'I-2', 'I-3', 'I-4')
    AND nt.valor_lab ='1' 
    AND egc.cod_ue !=0 --ESSALUD
 """)
    return gestante_captadas

def gestante_primer_trimestre(conn,anio,mes_ini,mes_fin):
    gestante_primer_trimestre=conn.df(f"""
SELECT DISTINCT 
    cod_eess,
    numero_documento,
    fecha_atencion,
    1 AS "gestantes_tri"
FROM 
    maestros.nominal_trama nt  
    INNER JOIN maestros.maestro_paciente mp  ON nt.id_paciente = mp.id_paciente
    INNER JOIN maestros.eess_geresa_cusco egc ON nt.id_establecimiento = egc.id_eess
WHERE 
    codigo_item IN ('Z3491', 'Z3591') 
    AND valor_lab = '1' 
    AND genero = 'F' 
    AND anio = {anio} 
    AND mes BETWEEN {mes_ini} AND {mes_fin} 
    AND id_tipo_documento = '1' 
    AND cat IN ('I-1', 'I-2', 'I-3', 'I-4')
    AND egc.cod_ue !=0 --ESSALUD
""")
    return gestante_primer_trimestre
def asignar_observaciones(st):
    if st >= 2:
        return 'REGISTRO OBSERVADO'
    else:
        return ''

In [36]:
gestante_captadas=gestante_captadas(conn,anio,mes_ini,mes_fin)
gestante_primer_trimestre=gestante_primer_trimestre(conn,anio,mes_ini,mes_fin)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\fed\../../..\clases\bd\conexion2.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [37]:
gestante_captadas

# Agregar unir 
fed = gestante_captadas.merge( gestante_primer_trimestre, on=['numero_documento', 'cod_eess'], how='left')

trimestre_map = {
    'Z3491': 'I',
    'Z3591': 'I',
    'Z3492': 'II',
    'Z3592': 'II'
}

# Agregar una nueva columna "trimestre" basada en el mapeo
fed['trimestre'] = fed['codigo_item'].map(trimestre_map).fillna('III')

fed.head()

,provincia,distrito,desc_ue,red,microred,nombre_eess,anio,mes,codigo_item,numero_documento,cod_eess,fechacaptada,valor_lab,gestantes,fecha_atencion,gestantes_tri,trimestre
0,ANTA,HUAROCONDO,UE - SALUD CUSCO NORTE,CUSCO NORTE,ANTA,HUAROCONDO,2024,2,Z3592,40033225,2333,2024-02-04,1,1,NaN,NaN,II
1,CANCHIS,COMBAPATA,UE - SALUD CANAS - CANCHIS - ESPINAR,CANAS-CANCHIS-ESPINAR,COMBAPATA,COMBAPATA CANCHIS,2024,1,Z3591,40199624,2386,2024-01-20,1,1,2024-01-20,1.0,I
2,CANAS,LANGUI,UE - SALUD CANAS - CANCHIS - ESPINAR,CANAS-CANCHIS-ESPINAR,TECHO OBRERO,LANGUI,2024,1,Z3591,40477407,2369,2024-01-25,1,1,2024-01-25,1.0,I
3,ANTA,ANTA,UE - SALUD CUSCO NORTE,CUSCO NORTE,ANTA,ANTA,2024,1,Z3591,40493377,2326,2024-01-10,1,1,2024-01-10,1.0,I
4,CUSCO,SAN SEBASTIAN,UE - SALUD CUSCO SUR,CUSCO SUR,CUSCO,SAN SEBASTIAN,2024,1,Z3591,40591689,2301,2024-01-08,1,1,2024-01-08,1.0,I


In [38]:
fed['den'] = (fed['gestantes'] == 1).astype(int)
fed['obs'] = fed['gestantes'].apply(asignar_observaciones)

fed = fed.drop(columns=['valor_lab', 'gestantes','codigo_item','cod_eess'])
fed = fed.rename(columns={'gestantes_tri': 'nun', 'numero_documento': 'dni'})

fed

,provincia,distrito,desc_ue,red,microred,nombre_eess,anio,mes,dni,fechacaptada,fecha_atencion,nun,trimestre,den,obs
0,ANTA,HUAROCONDO,UE - SALUD CUSCO NORTE,CUSCO NORTE,ANTA,HUAROCONDO,2024,2,40033225,2024-02-04,NaN,NaN,II,1,
1,CANCHIS,COMBAPATA,UE - SALUD CANAS - CANCHIS - ESPINAR,CANAS-CANCHIS-ESPINAR,COMBAPATA,COMBAPATA CANCHIS,2024,1,40199624,2024-01-20,2024-01-20,1.0,I,1,
2,CANAS,LANGUI,UE - SALUD CANAS - CANCHIS - ESPINAR,CANAS-CANCHIS-ESPINAR,TECHO OBRERO,LANGUI,2024,1,40477407,2024-01-25,2024-01-25,1.0,I,1,
3,ANTA,ANTA,UE - SALUD CUSCO NORTE,CUSCO NORTE,ANTA,ANTA,2024,1,40493377,2024-01-10,2024-01-10,1.0,I,1,
4,CUSCO,SAN SEBASTIAN,UE - SALUD CUSCO SUR,CUSCO SUR,CUSCO,SAN SEBASTIAN,2024,1,40591689,2024-01-08,2024-01-08,1.0,I,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,CUSCO,SAN JERONIMO,UE - SALUD CUSCO SUR,CUSCO SUR,CUSCO,SAN JERONIMO,2024,1,81603910,2024-01-18,2024-01-18,1.0,I,1,
1554,LA CONVENCION,MEGANTONI,UE - SALUD LA CONVENCION,LA CONVENCION,KAMISEA,KAMISEA,2024,1,82056698,2024-01-21,2024-01-21,1.0,I,1,
1555,LA CONVENCION,SANTA ANA,UE - SALUD LA CONVENCION,LA CONVENCION,SANTA ANA,SANTA ANA,2024,1,90000023,2024-01-30,NaN,NaN,II,1,
1556,CANCHIS,SICUANI,UE - SALUD CANAS - CANCHIS - ESPINAR,CANAS-CANCHIS-ESPINAR,PAMPAPHALLA,LA FLORIDA,2024,1,91242507,2024-01-18,2024-01-18,1.0,I,1,


In [39]:

conn.insert_df(fed,'fed_si_01_01','materno')


DataFrame insertado en la tabla fed_si_01_01
